In [1]:
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
train_folder='chest_xray/train'
val_folder='chest_xray/val'
test_folder='chest_xray/test'

In [4]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 200
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [5]:
train=get_data(train_folder)
val=get_data(val_folder)
test=get_data(test_folder)


C:\Users\VEENA\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [6]:
from sklearn.model_selection import train_test_split
X = []
y = []

for feature, label in train:
    X.append(feature)
    y.append(label)

for feature, label in test:
    X.append(feature)
    y.append(label)
    
for feature, label in val:
    X.append(feature)
    y.append(label)


X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=32)

In [7]:
X_train=X_train/255
X_test=X_test/255
X_val=X_val/255

In [8]:
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen =ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=90, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True)  

datagen.fit(X_train)


In [9]:
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
model=tf.keras.Sequential()

model.add(Conv2D(6, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization(axis=1))

model.add(Conv2D(4, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization(axis=1))

model.add(Conv2D(4, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization(axis=1))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.compile(optimizer='sgd', loss='mse')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 6)       60        
_________________________________________________________________
activation (Activation)      (None, 200, 200, 6)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 6)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100, 6)       400       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 4)       220       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 4)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 4)         0

In [12]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=10),validation_data=(X_val, y_val), epochs=15)

  ...
    to  
  ['...']
Train for 375 steps, validate on 937 samples
Epoch 1/15
375/375 [==============================] - 97s 259ms/step - loss: 0.2292 - val_loss: 0.1834
Epoch 2/15
375/375 [==============================] - 88s 234ms/step - loss: 0.1912 - val_loss: 0.1646
Epoch 3/15
375/375 [==============================] - 131s 350ms/step - loss: 0.1728 - val_loss: 0.1440
Epoch 4/15
375/375 [==============================] - 141s 375ms/step - loss: 0.1540 - val_loss: 0.1131
Epoch 5/15
375/375 [==============================] - 149s 396ms/step - loss: 0.1359 - val_loss: 0.1142
Epoch 6/15
375/375 [==============================] - 130s 347ms/step - loss: 0.1259 - val_loss: 0.0997
Epoch 7/15
375/375 [==============================] - 150s 399ms/step - loss: 0.1218 - val_loss: 0.0870
Epoch 8/15
375/375 [==============================] - 136s 362ms/step - loss: 0.1189 - val_loss: 0.0800
Epoch 9/15
375/375 [==============================] - 155s 414ms/step - loss: 0.1151 - val_loss: 0.1

In [13]:
model.evaluate(X_train, y_train)

3747/3747 [==============================] - 51s 14ms/sample - loss: 0.0729


0.07285201334483248

In [14]:
pred = model.predict(X_train)

In [15]:
predictions = model.predict(X_test)

In [16]:
binary_predictions = []
threshold = 0.5
for i in predictions:
    if i >= threshold:
        binary_predictions.append(1)
    else:
        binary_predictions.append(0) 

In [17]:
binary_predictions=np.array(binary_predictions)

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [19]:
print("Accuracy :",accuracy_score(binary_predictions,y_test))

Accuracy : 0.9052901023890785


In [22]:
model.save('pneumonia_detection5.h5')